In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [75]:
data = pd.read_csv('/kaggle/input/car-price-prediction/CarPrice_Assignment.csv')
data.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [76]:
data.drop('car_ID',axis=1,inplace=True)
data.drop('symboling',axis=1,inplace=True)
data.drop('CarName',axis=1,inplace=True)
data.drop('fueltype',axis=1,inplace=True)
data.drop('aspiration',axis=1,inplace=True)
data.drop('doornumber',axis=1,inplace=True)
data.drop('carbody',axis=1,inplace=True)
data.drop('drivewheel',axis=1,inplace=True)
data.drop('enginelocation',axis=1,inplace=True)
data.drop('fuelsystem',axis=1,inplace=True)
data.drop('enginetype',axis=1,inplace=True)
data['cylindernumber'].replace({'three':3,'four':4,'five':5,'six':6,'twelve':12,'eight':6,'two':2},inplace=True)
data['cylindernumber']=data['cylindernumber'].astype(str).astype(int)

In [77]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   wheelbase         205 non-null    float64
 1   carlength         205 non-null    float64
 2   carwidth          205 non-null    float64
 3   carheight         205 non-null    float64
 4   curbweight        205 non-null    int64  
 5   cylindernumber    205 non-null    int64  
 6   enginesize        205 non-null    int64  
 7   boreratio         205 non-null    float64
 8   stroke            205 non-null    float64
 9   compressionratio  205 non-null    float64
 10  horsepower        205 non-null    int64  
 11  peakrpm           205 non-null    int64  
 12  citympg           205 non-null    int64  
 13  highwaympg        205 non-null    int64  
 14  price             205 non-null    float64
dtypes: float64(8), int64(7)
memory usage: 24.1 KB


In [78]:
#Centralize Values
def toCentralizeInput(_data):
  for col in _data.columns:
    if(col != 'price'):
      _data[col] = [x - data[col].mean()/_data[col].std() for x in _data[col]]

toCentralizeInput(data)
data.head()

,wheelbase,carlength,carwidth,carheight,curbweight,cylindernumber,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,72.200089,154.692421,33.37667,26.813344,2543.091871,-0.543263,126.952471,-8.824013,-7.700885,6.446517,108.367069,4989.255186,17.145068,22.534528,13495.0
1,72.200089,154.692421,33.37667,26.813344,2543.091871,-0.543263,126.952471,-8.824013,-7.700885,6.446517,108.367069,4989.255186,17.145068,22.534528,16500.0
2,78.100089,157.092421,34.77667,30.413344,2818.091871,1.456737,148.952471,-9.614013,-6.910885,6.446517,151.367069,4989.255186,15.145068,21.534528,16500.0
3,83.400089,162.492421,35.47667,32.313344,2332.091871,-0.543263,105.952471,-9.104013,-6.980885,7.446517,99.367069,5489.255186,20.145068,25.534528,13950.0
4,83.000089,162.492421,35.67667,32.313344,2819.091871,0.456737,132.952471,-9.104013,-6.980885,5.446517,112.367069,5489.255186,14.145068,17.534528,17450.0


In [81]:
train_X = data.sample(frac=0.8,random_state=0)
val_X = data.drop(train_X.index)
test_X = train_X.sample(frac=0.3,random_state=0)

train_Y = train_X['price']
train_X.drop('price',axis=1,inplace=True)

val_Y = val_X['price']
val_X.drop('price',axis=1,inplace=True)

test_Y = test_X['price']
test_X.drop('price',axis=1,inplace=True)

In [93]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(10, activation='relu',
                       input_shape=(train_X.shape[1],)))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(6, activation='relu'))
model.add(layers.Dense(1))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

In [94]:
model.fit(train_X, train_Y,
              epochs=100, batch_size=1)

Epoch 1/100
164/164 [==============================] - 1s 1ms/step - loss: 243437131.3939 - mae: 13167.0747
Epoch 2/100
164/164 [==============================] - 0s 1ms/step - loss: 181408798.4727 - mae: 11324.7131
Epoch 3/100
164/164 [==============================] - 0s 1ms/step - loss: 78371672.6848 - mae: 6284.5715
Epoch 4/100
164/164 [==============================] - 0s 1ms/step - loss: 57043959.6545 - mae: 4851.5219
Epoch 5/100
164/164 [==============================] - 0s 1ms/step - loss: 47544010.9152 - mae: 4813.7057
Epoch 6/100
164/164 [==============================] - 0s 1ms/step - loss: 59363750.9697 - mae: 5678.0516
Epoch 7/100
164/164 [==============================] - 0s 1ms/step - loss: 51570258.2000 - mae: 5180.7933
Epoch 8/100
164/164 [==============================] - 0s 1ms/step - loss: 64268767.7030 - mae: 5542.7448
Epoch 9/100
164/164 [==============================] - 0s 1ms/step - loss: 44739879.7030 - mae: 4882.2855
Epoch 10/100
164/164 [====================

164/164 [==============================] - 0s 1ms/step - loss: 31803485.3333 - mae: 3720.5730TA: 0s - loss: 33585498.4000 - mae: 3840.226
Epoch 78/100
164/164 [==============================] - 0s 1ms/step - loss: 15939439.3503 - mae: 2575.8657
Epoch 79/100
164/164 [==============================] - 0s 1ms/step - loss: 27811906.2636 - mae: 3350.9718
Epoch 80/100
164/164 [==============================] - 0s 1ms/step - loss: 22166707.0455 - mae: 2936.4360
Epoch 81/100
164/164 [==============================] - 0s 1ms/step - loss: 16523810.4706 - mae: 2684.8807
Epoch 82/100
164/164 [==============================] - 0s 1ms/step - loss: 27088043.3492 - mae: 3143.1226
Epoch 83/100
164/164 [==============================] - 0s 1ms/step - loss: 19231544.6302 - mae: 2854.3804
Epoch 84/100
164/164 [==============================] - 0s 1ms/step - loss: 22384246.7818 - mae: 3171.5498
Epoch 85/100
164/164 [==============================] - 0s 1ms/step - loss: 12899190.6371 - mae: 2306.7847
Epoch 

In [95]:
val_mse, val_mae = model.evaluate(val_X, val_Y)

2/2 [==============================] - 0s 4ms/step - loss: 15976361.0000 - mae: 2401.2747


In [98]:
test_mse,test_mae = model.evaluate(test_X,test_Y)

2/2 [==============================] - 0s 3ms/step - loss: 6905128.5000 - mae: 1939.9272


In [99]:
test_mae

1939.92724609375

In [101]:
model.predict(test_X)

array([[14185.008 ],
       [ 8477.439 ],
       [ 7782.454 ],
       [ 9388.529 ],
       [16480.652 ],
       [10728.381 ],
       [ 8180.7686],
       [ 7695.976 ],
       [ 7985.464 ],
       [12322.756 ],
       [ 9830.606 ],
       [ 9433.913 ],
       [14260.351 ],
       [ 6911.79  ],
       [10881.191 ],
       [17700.334 ],
       [14526.2   ],
       [19460.434 ],
       [ 7054.864 ],
       [24100.96  ],
       [ 6827.838 ],
       [ 6703.0503],
       [11805.685 ],
       [19793.9   ],
       [12643.053 ],
       [19145.754 ],
       [20686.646 ],
       [17648.947 ],
       [ 8872.352 ],
       [ 6523.786 ],
       [ 5652.654 ],
       [13703.432 ],
       [ 8422.16  ],
       [18447.178 ],
       [21272.29  ],
       [27302.734 ],
       [ 7347.7217],
       [10242.654 ],
       [ 6889.7275],
       [12054.471 ],
       [ 8980.734 ],
       [12643.053 ],
       [ 6392.348 ],
       [10598.332 ],
       [11023.01  ],
       [11700.924 ],
       [ 8628.693 ],
       [ 8537

In [103]:
test_Y[0]

13495.0

In [ ]:
##End